In [2]:
library(regioneR)

Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb



In [4]:
map_bed <- toGRanges(read.table('../Data/mappability.stranded.bed', sep="\t"))

In [3]:
cd19_bed <- toGRanges(read.table('../Data/interval.all.bed', sep="\t"))

In [4]:
cd19_bed

GRanges object with 86147394 ranges and 0 metadata columns:
           seqnames            ranges strand
              <Rle>         <IRanges>  <Rle>
         1     chr1     180788-180789      *
         2     chr1     180789-180790      *
         3     chr1     180790-180791      *
         4     chr1     180791-180792      *
         5     chr1     180792-180793      *
       ...      ...               ...    ...
  86147390     chrY 56838579-56838580      *
  86147391     chrY 56838580-56838581      *
  86147392     chrY 56838581-56838582      *
  86147393     chrY 56838582-56838583      *
  86147394     chrY 56838583-56838584      *
  -------
  seqinfo: 24 sequences from an unspecified genome; no seqlengths